In [19]:
# -*- coding: utf-8 -*-
"""
Inference of the MRI-SAM on the nifti datasets.
"""
import os
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import functional as F
import torch
from utils.losses import DiceFocalLoss, DiceLoss

In [20]:
os.getcwd()

'/Users/wangzhao/Library/CloudStorage/OneDrive-TheUniversityofQueensland/PhD/CodePractice/SAMRI'

In [ ]:
def get_preprocess_shape(oldh: int, oldw: int, long_side_length: int) :
        """
        Compute the output size given input size and target long side length.
        """
        scale = long_side_length * 1.0 / max(oldh, oldw)
        newh, neww = oldh * scale, oldw * scale
        neww = int(neww + 0.5)
        newh = int(newh + 0.5)
        return (newh, neww)

In [ ]:
def preprocess_mask(mask, target_size=256):
    """
    Preprocess masks from original size to target size.

    Args:
        mask (np.array): the mask with shape of BxCxHxW
        target_size (int, optional): The target size. Defaults to 256.
        
    Returns:
        (np.array): the mask with the target shape.
        
    """
    resize_long = get_preprocess_shape(mask.shape[-2], mask.shape[-1], target_size)
    print(resize_long)
    resized_mask = F.interpolate(mask, 
                           resize_long, 
                           mode="nearest")
    # Pad
    print(resized_mask.shape)
    h, w = resized_mask.shape[-2:]
    padh = target_size - h
    padw = target_size - w
    x = F.pad(resized_mask, (0, padw, 0, padh), value=0)
    print(x.shape)
    return x

In [21]:
mask = torch.ones((128,255))[None,None,:,:]
mask1 = torch.zeros((128,255))[None,None,:,:]

In [22]:
mask1.shape

torch.Size([1, 1, 128, 255])

In [45]:
loss1 = DiceFocalLoss(sigmoid=False, squared_pred=True, batch= True, reduction="mean",lambda_dice=1,lambda_focal=10)

loss2 = DiceLoss(sigmoid=False, squared_pred=True,batch= True, reduction="mean")

loss3 = DiceFocalLoss(sigmoid=True, squared_pred=True,reduction="mean",lambda_dice=1,lambda_focal=10)

loss4 = DiceLoss(sigmoid=True, squared_pred=True,reduction="mean")

In [46]:
print(loss1(mask,mask), loss2(mask,mask), loss3(mask,mask), loss4(mask,mask))

tensor(0.2266) tensor(0.) tensor(0.2737) tensor(0.0471)


In [36]:
print(loss1(mask,mask1), loss2(mask,mask1), loss3(mask,mask1), loss4(mask,mask1))

tensor(8.0187) tensor(1.) tensor(8.0187) tensor(1.)


In [ ]:
get_preprocess_shape(mask.shape[0], mask.shape[1], 256)

In [ ]:
re = preprocess_mask(mask.type(torch.float)[None,None,:,:],target_size=256)

In [ ]:
re

In [ ]:
re.shape